# E-classify-train: This Notebook trains the NOIRE-Net E-region classification networks

## 1 - Import libaries 

In [1]:
import os
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import pickle
from sklearn.model_selection import KFold

## 2 - Define function to get ionogram label (True = E-region, False = No E-region)

In [2]:
# The function get_label_from_par reads a .par file and returns 
# True if either the second or fourth item is not 'nan', 
# indicating the presence of an E-region in the corresponding image,
# for use in binary classification.

def get_label_from_par(par_file_path):
    try:
        # Open the .par file located at par_file_path
        with open(par_file_path, 'r') as file:
            content = file.readline().strip()  # Read and strip the first line of the file
            items = content.split()  # Split the line into individual items (usually numbers or 'nan')

            # Check the second (index 1) and fourth (index 3) items:
            # If either of these items is not 'nan', it implies the presence of an E-region.
            # The function then returns True, indicating that this image has an E-region.
            is_e_region = items[1].lower() != 'nan' or items[3].lower() != 'nan'
        return is_e_region

    except IndexError:
        # This block catches an IndexError, which occurs if the line read from the file
        # does not have at least four items. In such a case, the function assumes that
        # the E-region data is not present or not properly formatted, and returns False.
        return False

    except Exception as e:
        # This block catches any other exceptions (like file not found, read errors, etc.).
        # It prints the exception message and returns False, indicating an issue with processing the file.
        print(f"Error reading {par_file_path}: {e}")
        return False

## 3 - Define function to load ionograms and preprocess the data

In [3]:
# The load_data function loads and preprocesses images and their corresponding labels
# from specified directories, based on a list of file identifiers, and returns arrays
# of these images and labels for further processing.

# Function to load the images and labels given a list of file bases (timestamps)
def load_data(file_bases, ionogram_dir, parameter_dir):
    images = []  # List to store preprocessed images
    labels = []  # List to store corresponding labels
    
    # Iterate over the provided file bases to load images and labels
    for base in file_bases:
        img_path = os.path.join(ionogram_dir, base + '.png')
        par_path = os.path.join(parameter_dir, base + '.par')

        # Load the image, convert it to grayscale, resize it, and normalize pixel values
        image = load_img(img_path, color_mode='grayscale', target_size=(310, 310))
        image = img_to_array(image)
        image /= 255.0  # Normalize image pixels to be between 0 and 1

        # Load the label for the image using the get_label_from_par function
        label = get_label_from_par(par_path)

        # Append the preprocessed image and label to their respective lists
        images.append(image)
        labels.append(label)

    # Convert the lists of images and labels to numpy arrays
    return np.array(images), np.array(labels).astype(int)

## 4 - Define data paths and sort the files by their epoch timestamp

In [4]:
# This code defines paths to directories containing ionogram images and parameter files, 
# then creates a sorted list of unique identifiers (epoch timestamps) from the names of the ionogram images, 
# ensuring the data is ordered chronologically.

# Define the directory paths to the data
ionogram_dir = '/Users/akv020/Tensorflow/NOIRE-Net/data/train-val-test/ionograms'
parameter_dir = '/Users/akv020/Tensorflow/NOIRE-Net/data/train-val-test/parameters'

# Get a sorted list of file names (without file extensions) from the ionogram directory
data_files = sorted([f.split('.')[0] for f in os.listdir(ionogram_dir) if f.endswith('.png')])

# Ensure the data is sorted by the epoch timestamp
data_files = sorted(data_files, key=int)

## 5 - Define training, validation and testing indeces for 10-fold cross-validation run

In [5]:
# This code sets up a 10-fold cross-validation framework, pre-computing and storing indices for 
# training, validation, and test sets for each fold, with a circular permutation applied 
# to the test set indices to maintain the temporal order of the data.

# Define the number of splits for KFold
n_splits = 10
kf = KFold(n_splits=n_splits)

# Pre-compute the indices for training, validation, and test sets for each fold
train_indices = []
val_indices = []
test_indices = []

# Pre-compute the indices for training, validation, and test sets for each fold
train_indices = []
val_indices = []
test_indices = []

# Number of samples for validation and test sets
num_val_samples = int(len(data_files) * 0.1)

for fold, (train_val_idx, test_idx) in enumerate(kf.split(data_files)):
    # Circular permutation for the test set
    test_idx = np.roll(test_idx, fold * num_val_samples)
    
    # Split the train_val_idx further into train and validation sets
    val_idx = train_val_idx[:num_val_samples]
    train_idx = train_val_idx[num_val_samples:]
    
    # Store the indices
    train_indices.append(train_idx)
    val_indices.append(val_idx)
    test_indices.append(test_idx)

## 6 - Define a function to create the NOIRE-Net architecture

In [6]:
# This code defines and complies NOIRE-Net a convolutional neural network (CNN) model using Keras, 
# with multiple convolutional layers, batch normalization, max pooling, and dense layers, 
# designed for binary classification tasks.

def NOIREnet():
    model = Sequential([
    # First convolutional layer with 32 filters and a kernel size of 3x3
    # 'padding=same' ensures the output size is the same as the input size
    # 'input_shape' is set for the first layer to indicate the shape of the input data
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(310, 310, 1)),
    
    # Batch normalization to normalize the activations from the previous layer
    BatchNormalization(),

    # Second convolutional layer with 32 filters and a kernel size of 3x3
    Conv2D(32, (3, 3), activation='relu'),

    # Another batch normalization
    BatchNormalization(),

    # First max pooling layer to reduce spatial dimensions
    MaxPooling2D((2, 2)),

    # Repeating the pattern of two convolutional layers followed by batch normalization
    # and a max pooling layer, gradually increasing the number of filters
    Conv2D(32, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Flatten the output from the convolutional layers to feed into dense layers
    Flatten(),

    # Dense (fully connected) layer with 256 neurons and relu activation
    Dense(256, activation='relu'),

    # Dropout layer to reduce overfitting
    Dropout(0.5),

    # Another dense layer with 128 neurons
    Dense(128, activation='relu'),

    # Output layer with a single neuron and sigmoid activation for binary classification
    Dense(1, activation='sigmoid')
    ])
    
    # Compile the CNN model
    model.compile(
        optimizer='adam',  # Using the Adam optimizer for adaptive learning rate optimization
        loss='binary_crossentropy',  # Binary crossentropy loss function, suitable for binary classification tasks
        metrics=['accuracy']  # The model will report 'accuracy' as a performance metric
    )
    
    # Return the compiled model
    return model

## 7 - Train 10 CNNs for E-region classification and save the models

In [7]:
# This code trains 10 Convolutional Neural Networks (CNNs) on different folds of the
# dataset for binary classification, saves the best model of each training session, 
# and records their training histories.


# Create the directory for saving models and histories if it doesn't exist
save_dir = '/Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-classify'
os.makedirs(save_dir, exist_ok=True)

# Initialize lists to store the training histories and filenames of the best models
histories = []
model_filenames = []

# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,      # Factor to reduce the learning rate
    patience=6,     # Number of epochs with no improvement to wait before reducing LR
    min_lr=0.00001   # Minimum learning rate
)

# Loop to train 10 CNN models over 10 folds
for i in range(n_splits):
    
    # Select the pre-computed indices for the current fold
    train_idx = train_indices[i]
    val_idx = val_indices[i]
    test_idx = test_indices[i]
    
    # Convert indices to file names for loading data
    train_files = [data_files[idx] for idx in train_idx]
    val_files = [data_files[idx] for idx in val_idx]
    test_files = [data_files[idx] for idx in test_idx]
    
    # Load the data for the current fold
    X_train, y_train = load_data(train_files, ionogram_dir, parameter_dir)
    X_val, y_val = load_data(val_files, ionogram_dir, parameter_dir)
    # X_test, y_test = load_data(test_files, ionogram_dir, parameter_dir) Not needed during the training stage
    
    # Create a new CNN model for each iteration
    model = NOIREnet()

    # Define the filename for the checkpoint model
    model_filename = os.path.join(save_dir, f'E-classify_run{i+1}.h5')

    # Define a checkpoint callback to save the best model based on validation accuracy
    checkpoint_callback = ModelCheckpoint(
        model_filename,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='max',
        save_weights_only=False
    )

    # Train the model with specified callbacks including ReduceLROnPlateau
    history = model.fit(
        X_train, y_train,
        batch_size=64,
        epochs=30,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint_callback, reduce_lr]  # Include ReduceLROnPlateau callback
    )

    # Save the training history and the filename of the saved best model
    histories.append(history.history)
    model_filenames.append(model_filename)
    
    print(f"Completed fold {i+1}/{n_splits}")

# Optionally, save the training histories to a file in the same 'E-classify' directory
history_filename = os.path.join(save_dir, 'training_histories.pkl')
with open(history_filename, 'wb') as file:
    pickle.dump({'histories': histories, 'model_filenames': model_filenames}, file)


Metal device set to: Apple M1 Max


2023-12-14 23:34:57.492920: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-14 23:34:57.493028: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/30


2023-12-14 23:34:58.585499: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-14 23:34:59.244195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x64x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be

209/210 [============================>.] - ETA: 0s - loss: 0.6203 - accuracy: 0.6187

...

210/210 [==============================] - ETA: 0s - loss: 0.6037 - accuracy: 0.6679

2023-12-15 05:43:50.844318: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x64x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x64x1x1xi1>'



Epoch 1: val_accuracy improved from -inf to 0.30411, saving model to /Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-classify/E-classify_run10.h5
210/210 [==============================] - 73s 331ms/step - loss: 0.6037 - accuracy: 0.6679 - val_loss: 2.7622 - val_accuracy: 0.3041 - lr: 0.0010
Epoch 2/30


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x13x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/495c257e-668e-11ee-93ce-926038f30c31/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x13x1x1xi1>'


210/210 [==============================] - ETA: 0s - loss: 0.5749 - accuracy: 0.6885
Epoch 2: val_accuracy improved from 0.30411 to 0.71497, saving model to /Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-classify/E-classify_run10.h5
210/210 [==============================] - 70s 333ms/step - loss: 0.5749 - accuracy: 0.6885 - val_loss: 0.5558 - val_accuracy: 0.7150 - lr: 0.0010
Epoch 3/30
210/210 [==============================] - ETA: 0s - loss: 0.5634 - accuracy: 0.6923
Epoch 3: val_accuracy improved from 0.71497 to 0.91234, saving model to /Users/akv020/Tensorflow/NOIRE-Net/NOIRE-Net/E-classify/E-classify_run10.h5
210/210 [==============================] - 69s 328ms/step - loss: 0.5634 - accuracy: 0.6923 - val_loss: 0.2504 - val_accuracy: 0.9123 - lr: 0.0010
Epoch 4/30
210/210 [==============================] - ETA: 0s - loss: 0.5647 - accuracy: 0.6924
Epoch 4: val_accuracy did not improve from 0.91234
210/210 [==============================] - 69s 330ms/step - loss: 0.5647 - accurac